In [2]:
import json
import pprint
import pandas as pd
import pymysql
from watson_developer_cloud import NaturalLanguageUnderstandingV1
import watson_developer_cloud.natural_language_understanding.features.v1 as Features
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+') # to remove punctuations

#connecting to watson developer cloud. Setting up for API call on IBM watson's natural language understanding

In [16]:
natural_language_understanding = NaturalLanguageUnderstandingV1(
  username="0fae1b36-6da0-48ed-a19a-aab68667aa21",
  password="HSytcyR8sY4q",
  version="2017-02-27")

In [131]:
txt="Wouldn't recommend for anyone who has problems with anxiety. I never had a panic attack after smoking weed until I smoked Al's Dream. Not sure if it was fully the fault of the strain or what but I definitely will not be trying this strain again"

In [133]:
response = natural_language_understanding.analyze(
                                                  text=txt,
                                                  features=[Features.Keywords(),
                                                            Features.Emotion(targets=["strain"]),
                                                            Features.Sentiment(targets=["strain"]),
                                                            ],
                                                  language='en'
                                                 )
pprint.pprint(response)

{'emotion': {'document': {'emotion': {'anger': 0.123642,
                                      'disgust': 0.112105,
                                      'fear': 0.877317,
                                      'joy': 0.009952,
                                      'sadness': 0.121765}},
             'targets': [{'emotion': {'anger': 0.248962,
                                      'disgust': 0.172239,
                                      'fear': 0.153963,
                                      'joy': 0.069943,
                                      'sadness': 0.272133},
                          'text': 'strain'}]},
 'keywords': [{'relevance': 0.984396, 'text': 'panic attack'},
              {'relevance': 0.865892, 'text': 'strain'},
              {'relevance': 0.68718, 'text': 'anxiety'},
              {'relevance': 0.680216, 'text': 'fault'},
              {'relevance': 0.652196, 'text': 'weed'},
              {'relevance': 0.579098, 'text': 'problems'},
              {'relevance': 0.5

Generating response.
Look into - https://www.ibm.com/watson/developercloud/natural-language-understanding/api/v1/#authentication

#retrivng customer reviews stored on mysql database into a pandas dataframe

In [11]:
connection =pymysql.connect(
                            host='***',
                            user='dev1',
                            passwd='****',
                            db='***')
sql='select * from leafly_strainreviews'
data=pd.read_sql(sql,con=connection)


In [49]:
df=pd.read_csv("hotel_reviews.csv")

In [50]:
df.head(3)

,address,categories,city,country,latitude,longitude,name,postalCode,province,reviews.date,reviews.dateAdded,reviews.doRecommend,reviews.id,reviews.rating,reviews.text,reviews.title,reviews.userCity,reviews.username,reviews.userProvince
0,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-09-22T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,4.0,Pleasant 10 min walk along the sea front to th...,Good location away from the crouds,NaN,Russ (kent),NaN
1,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-04-03T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Really lovely hotel. Stayed on the very top fl...,Great hotel with Jacuzzi bath!,NaN,A Traveler,NaN
2,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2014-05-13T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Ett mycket bra hotell. Det som drog ner betyge...,Lugnt l��ge,NaN,Maud,NaN


In [51]:
df.shape

(35912, 19)

analyzing the revirews

In [60]:
df1=df[0:10]
df1=df1[["reviews.text"]]
df1

,reviews.text
0,Pleasant 10 min walk along the sea front to th...
1,Really lovely hotel. Stayed on the very top fl...
2,Ett mycket bra hotell. Det som drog ner betyge...
3,We stayed here for four nights in October. The...
4,We stayed here for four nights in October. The...
5,We loved staying on the island of Lido! You ne...
6,Lovely view out onto the lagoon. Excellent vie...
7,ottimo soggiorno e ottima sistemazione nei gio...
8,Gnstiger Ausgangspunkt fr Venedig Besuche. Ruh...
9,Lidoen er perfekt til et par dages ro og afsla...


In [61]:
anger=[]
disgust=[]
fear=[]
joy=[]
sadness=[]
keywords=[]
sentiment=[]
sentiment_score=[]
count=0
for a in range(0,df1.shape[0]):
    txt=df.iloc[a]['reviews.text']
    count=count+1
    if len(txt)<20:
        sentiment.append('null')
        sentiment_score.append('null')
        anger.append("null")
        disgust.append("null")
        fear.append("null")
        joy.append("null")
        sadness.append("null")
        keywords.append("null")
        continue
    response = natural_language_understanding.analyze(
                                                      text=txt,
                                                      features=[
                                                                 Features.Keywords(),
                                                                 Features.Emotion(),
                                                                 Features.Sentiment()],
                                                      language='en'
                                                     )
    sentiment.append(response['sentiment']['document']['label'])
    sentiment_score.append(response['sentiment']['document']['score'])
    anger.append(response['emotion']['document']['emotion']['anger'])
    disgust.append(response['emotion']['document']['emotion']['disgust'])
    fear.append(response['emotion']['document']['emotion']['fear'])
    joy.append(response['emotion']['document']['emotion']['joy'])
    sadness.append(response['emotion']['document']['emotion']['sadness'])
    words=[]
    for aa in response['keywords']:
        words.append(aa['text'])
    keywords.append(words)

In [62]:
df1['keywords'] = keywords
df1['sentiment']= sentiment
df1['sentiment_score']= sentiment_score
df1['joy'] = joy
df1['sadness'] = sadness
df1['fear'] = fear
df1['disgust'] = disgust
df1['anger'] = anger

In [63]:
df1.head()

,reviews.text,keywords,sentiment,sentiment_score,joy,sadness,fear,disgust,anger
0,Pleasant 10 min walk along the sea front to th...,"[Water Bus. restaurants, Room aircon, min, Ple...",negative,-0.130910,0.705191,0.048357,0.118621,0.164352,0.024069
1,Really lovely hotel. Stayed on the very top fl...,"[lovely hotel, Jacuzzi bath, Great location, g...",positive,0.516788,0.930877,0.011671,0.006477,0.021599,0.030317
2,Ett mycket bra hotell. Det som drog ner betyge...,"[fick ett rum, Ett mycket bra, det endast var,...",neutral,0.000000,0.263506,0.179896,0.082685,0.075268,0.068068
3,We stayed here for four nights in October. The...,"[airy rooms, hotel staff, nights, Beds, Plenty...",positive,0.573835,0.697699,0.065437,0.070148,0.045702,0.050515
4,We stayed here for four nights in October. The...,"[airy rooms, hotel staff, nights, Beds, Plenty...",positive,0.573835,0.697699,0.065437,0.070148,0.045702,0.050515


In [140]:
df.to_csv('reviews_ibm_watson_ptsd.csv',index=False)